# Demo 4 — Model Registry

Register models, assign aliases (`@champion`), load by alias, and simulate a rollback between versions.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
import mlflow
import warnings
warnings.filterwarnings("ignore")
from features import fourier_features

In [2]:
mlflow.set_tracking_uri("http://localhost:5050")
mlflow.set_experiment("temperature-forecast-trial")

MODEL_NAME = "temperature-forecast-trial"

In [3]:
df = pd.read_csv("../data/jena_daily_temp.csv", parse_dates=["Date Time"], index_col="Date Time")

train = df.iloc[:-90]
test = df.iloc[-90:]

ref_date = df.index[0]
exog_train = fourier_features(train.index, ref_date)
exog_test = fourier_features(test.index, ref_date)

## Train a good model & register as v1

In [4]:
with mlflow.start_run(run_name="registry-v1-good") as run:
    order = (5, 1, 2)
    model = ARIMA(train["temperature"], order=order, exog=exog_train)
    results = model.fit()

    forecast = results.forecast(steps=len(test), exog=exog_test)
    rmse = np.sqrt(mean_squared_error(test["temperature"], forecast))

    mlflow.log_param("order", order)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("aic", results.aic)

    model_info = mlflow.statsmodels.log_model(results, name="arima_model")

    # Register the model
    mv1 = mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/arima_model",
        name=MODEL_NAME,
    )
    print(f"Registered v{mv1.version} | RMSE: {rmse:.2f}")

Successfully registered model 'temperature-forecast-trial'.
2026/02/26 16:16:33 WARNING mlflow.tracking._model_registry.fluent: Run with id d89774256f984481bd90282fe1b81a2a has no artifacts at artifact path 'arima_model', registering model based on models:/m-a6f9feb7bc484ede96e73d5a14091388 instead
2026/02/26 16:16:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: temperature-forecast-trial, version 1


Registered v1 | RMSE: 3.98
🏃 View run registry-v1-good at: http://localhost:5050/#/experiments/5/runs/d89774256f984481bd90282fe1b81a2a
🧪 View experiment at: http://localhost:5050/#/experiments/5


Created version '1' of model 'temperature-forecast-trial'.


## Assign `@champion` alias to v1

In [5]:
client = mlflow.MlflowClient()

client.set_registered_model_alias(MODEL_NAME, "champion", mv1.version)
print(f"Alias @champion → v{mv1.version}")

Alias @champion → v1


## Load the model by alias and predict

In [6]:
champion = mlflow.statsmodels.load_model(f"models:/{MODEL_NAME}@champion")

# Generate Fourier features for the forecast horizon
forecast_dates = pd.date_range(start=train.index[-1] + pd.Timedelta(days=1), periods=30, freq="D")
exog_forecast = fourier_features(forecast_dates, ref_date)

forecast = champion.forecast(steps=30, exog=exog_forecast)
print("30-day forecast from @champion:")
print(forecast.values[:5], "...")

30-day forecast from @champion:
[13.12626917 13.15917272 13.18607907 12.97360015 12.93427807] ...


## Train a worse model & register as v2

In [7]:
with mlflow.start_run(run_name="registry-v2-worse") as run:
    order = (1, 0, 0)
    model = ARIMA(train["temperature"], order=order, exog=exog_train)
    results = model.fit()

    forecast = results.forecast(steps=len(test), exog=exog_test)
    rmse = np.sqrt(mean_squared_error(test["temperature"], forecast))

    mlflow.log_param("order", order)
    mlflow.log_metric("rmse", rmse)
    mlflow.log_metric("aic", results.aic)

    mlflow.statsmodels.log_model(results, name="arima_model")

    mv2 = mlflow.register_model(
        model_uri=f"runs:/{run.info.run_id}/arima_model",
        name=MODEL_NAME,
    )
    print(f"Registered v{mv2.version} | RMSE: {rmse:.2f} (worse!)")

Registered model 'temperature-forecast-trial' already exists. Creating a new version of this model...
2026/02/26 16:17:46 WARNING mlflow.tracking._model_registry.fluent: Run with id c26a0d5828ec4412b6384661ef6e3010 has no artifacts at artifact path 'arima_model', registering model based on models:/m-ebb8b7a4deca44089660b53f7347e90f instead
2026/02/26 16:17:46 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: temperature-forecast-trial, version 2


Registered v2 | RMSE: 3.69 (worse!)
🏃 View run registry-v2-worse at: http://localhost:5050/#/experiments/5/runs/c26a0d5828ec4412b6384661ef6e3010
🧪 View experiment at: http://localhost:5050/#/experiments/5


Created version '2' of model 'temperature-forecast-trial'.


## Simulate rollback: move `@champion` to v2, then back to v1

In [8]:
# Point @champion to v2 (bad idea!)
client.set_registered_model_alias(MODEL_NAME, "champion", mv2.version)
print(f"Alias @champion → v{mv2.version} (the worse model)")

# Verify
info = client.get_model_version_by_alias(MODEL_NAME, "champion")
print(f"Current @champion is v{info.version}")

Alias @champion → v2 (the worse model)
Current @champion is v2


In [9]:
# Rollback: point @champion back to v1
client.set_registered_model_alias(MODEL_NAME, "champion", mv1.version)
print(f"Rolled back! @champion → v{mv1.version}")

info = client.get_model_version_by_alias(MODEL_NAME, "champion")
print(f"Current @champion is v{info.version}")

Rolled back! @champion → v1
Current @champion is v1


---
**Next:** Open the MLflow UI at http://localhost:5050, go to the **Models** tab, and explore the registered versions and aliases.